In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from modules import AEC_builder

_encoder = AEC_builder.Encoder(input_shape=(2000, 16, 1), nb_layer=2)
_encoder.build()

if not(_encoder.error_shape):
    model = _encoder.model
    # model.summary()

    _bottle_neck = AEC_builder.BottleNeck(_encoder=_encoder.model)
    _bottle_neck.build()
    model2 = _bottle_neck.model
    model2.summary()
    # shape = model.output.shape
    # a = Flatten()(model.output)
    # a = Dense(units=shape[1]*shape[2]*shape[3])(a)
    # model2 = Model(model.input, a)
    # model2.summary()


In [ ]:
a = [1,3]
for i, ai in enumerate(a):
    print(i, ai)

In [ ]:
import numpy as np
int(2.7)

In [ ]:
from modules.AEC_builder import check_shape
from tensorflow.keras.layers import Dense, Flatten, Input

input = Input(shape=(2000, 16, 1))

print(check_shape(layer=input, nb_layer=4, conv_kernel=(10,2), conv_stride=(2,1), pool_kernel=(5,2), pool_stride=(2,2)))

In [ ]:
def show(**params):
    print(params.setdefault('a', (1,2)))

show()